In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import nibabel as nib
import glob
import math
from nilearn.image import resample_img
from nilearn import plotting
# get_ipython().magic(u'matplotlib inline')
import statistics
import math
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
cd /projects/hulacon/shared/neu_data/AVFP 


/gpfs/projects/hulacon/shared/neu_data/AVFP


In [3]:
#loop through a subset of mask number range
# mask_range = range(0,int(np.max(full_mask)))
mask_range = [2]+list(range(4,12))
mask_range = [0,1,3]
matrix_average_duration = 20

#atlas directory
# mask_fs = ['/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-maxprob-thr25-1mm.nii.gz', '/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-maxprob-thr25-1mm.nii.gz']
# mask_label_fs = ['/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-labels.csv', '/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-labels.csv']
mask_fs = ['/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-maxprob-thr25-1mm.nii.gz']
mask_label_fs = ['/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-labels.csv']
# mask_fs = ['/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-maxprob-thr25-1mm.nii.gz']
# mask_label_fs = ['/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-labels.csv']


In [4]:
logfile_folder = 'AVFP_logfiles_new'
data_folder = 'AVFP_NTFA_data'
data_suffix = '_AVFP_denoised_novideoregs'
log_header = ['Video_name', 'video_number', 'video_category(1-H 2-SO 3-SP)', 'novel_vs_familiar(1-N,2-Old)', 'run_number', 'video_onset', 'video_offset', 'video_duration_method1', 'video_duration_method2', 'Fear_rating_onset', 'fear_rating', 'fear_rating_RT', 'Arousal_rating_onset', 'arousal_rating', 'arousal_rating_RT', 'Valence_rating_onset', 'valence_rating', 'valence_rating_RT']
#get subject name from log file names
subj_dir = ['sub_'+i.split('/')[-2] for i in glob.glob(data_folder+'/*/*_run-1'+data_suffix+'.nii.gz')]
subj_dir.sort()
print(len(subj_dir))
save_dir = '/projects/hulacon/shared/neu_data/AVFP/ISECmem_RSA_ROI_analysis'+data_suffix

TR=0.8
num_TR = 600 #per run
TR_series = [x*0.8 for x in range(0, num_TR)]
TR_BOLD_offset = 7
##get timing info
# subj=subj_dir[1]
# subj_log_f = glob.glob(os.path.join(logfile_folder,'AffVidsMem*'+subj+'.txt'))[0]
# subj_log = pd.read_table(subj_log_f, sep=' ', header=None, names = log_header, index_col=False)
# #mean video TR duration
# video_duration = np.mean(subj_log['video_duration_method2'])
# #mean of first two ratings TR duration in TR
# rating_duration = np.mean([np.mean(subj_log['Arousal_rating_onset']-subj_log['video_offset']), np.mean(subj_log['Valence_rating_onset']-subj_log['Arousal_rating_onset'])])
# #mean of tiral duations in TR and mean of ITI in TR
# duration=[]
# ITI=[]
# for start_n,start in enumerate(subj_log['video_onset'][:-1].tolist()):
#     duration.append(subj_log['video_onset'].tolist()[start_n+1] - start)
#     ITI.append(subj_log['video_onset'].tolist()[start_n+1] - (start+video_duration+(rating_duration*3)))
# trial_duration = np.mean([i for i in duration if i >0])
# ITI_duration=np.mean([i for i in ITI if i >0])
TR_video_duration = 26 #video_duration/0.8=25.6 round to 26
TR_mean_rating= 5.3*3 #rating_duration/0.8=5.3 
TR_mean_mean_ITI = 6 #ITI_duration/0.8=5.8 round to 6
TR_trial_end = TR_video_duration+TR_mean_rating+TR_mean_mean_ITI
TR_mean_trial_duration = 60 #TR_mean_mean_ITI+TR_video_duration+TR_mean_rating+TR_mean_mean_ITI =53.9 round to 55
num_vid = pd.read_table(glob.glob(os.path.join(logfile_folder,'AffVidsMem*'+subj_dir[0]+'.txt'))[0], sep=' ', header=None, names = log_header, index_col=False).shape[0]

#sample subj data img for resampling
subj_data_img = nib.load(glob.glob(os.path.join(data_folder,subj_dir[0][-3:],'sub-'+subj_dir[0][-3:]+'_run-1'+data_suffix+'.nii.gz'))[0])

#find the index of the closest number to K in a list
def closest(lst, K): 
    val = lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))] 
    idx = lst.index(val)
    return idx

#use index of finding value x in 2D array (b) of value x as index of the first 2 dimensions for 3D array (a)
def select3Dfrom2D(a,b,x):
    b=old_vid.astype(int)
    
    new_array = []
    for dim1 in list(set(np.where(b==x)[0])):
        new_array.append(a[dim1,np.where(b==x)[1][np.where(b==x)[0]==dim1],:])
    new_array = np.array(new_array)
    
    return new_array

#use index of finding value x in 2D array (b) of value x as index of the same 2 dimensions for 2D array (a)
def select2Dfrom2D(a,b,x):
    new_array = np.empty(((sum(b==x)[0],) + (a.shape[1],)), dtype=object)
    for dim2N, dim2 in enumerate(list(set(np.where(b==x)[1]))):
        new_array[:, dim2N] = a[np.where(b==x)[0][np.where(b==x)[1]==dim2], dim2]
    new_array = np.array(new_array)
    return new_array

#use index of finding value x in 2D array (b) of value x as index of the LAST 2 dimensions for 4D array (a)
def select4Dfrom2D(a,b,x):
    new_array = np.empty((a.shape[0:2] + (sum(b==x)[0],) + (b.shape[1],)), dtype=object)
    for dim2N, dim2 in enumerate(list(set(np.where(b==x)[1]))):
        new_array[:, :, :, dim2N] = a[:, :, np.where(b==x)[0][np.where(b==x)[1]==dim2], dim2]
    new_array = np.array(new_array)
    return new_array


67


In [5]:
#combine
# #atlas directory
# mask_f = '/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-maxprob-thr25-1mm.nii.gz'
# # regions = [48, 20, 19, 5, 13, 22, 23, 35, 38, 39, 40]#new>old
# regions = [1, 2, 4, 12, 20, 21, 26, 28, 29, 32, 41, 42, 44]#old>new
# atlas_img = nib.load(mask_f)
# atlas_data = atlas_img.get_fdata()
# mask = atlas_data.copy()
# mask[:]=0
# for r in regions:
#     mask[atlas_data==r]=1
    
# # mask_f = '/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-maxprob-thr25-1mm.nii.gz'
# # regions = [19,20] #new>old
# # atlas_img = nib.load(mask_f)
# # atlas_data = atlas_img.get_fdata()
# # for r in regions:
# #     mask[atlas_data==r]=1
    
# mask_img = nib.Nifti1Image(mask, atlas_img.affine, atlas_img.header)
# # nib.save(mask_img, save_dir+'/pos_new_old.nii.gz')
# nib.save(mask_img, save_dir+'/neg_old_new.nii.gz')


In [6]:
#change to 50, because only plot 5 to -5 in the recorded time course
TR_mean_trial_duration = 50 
# TR_mean_trial_duration = 30 
start_TR = -2
line_ticks_range = x = np.arange(start_TR, TR_mean_trial_duration+start_TR)
matrix_ticks_range = x = np.arange(start_TR, TR_mean_trial_duration+start_TR, 2)
matshow_extent=[start_TR,TR_mean_trial_duration+start_TR,TR_mean_trial_duration+start_TR,start_TR]


######################################################################

CODE BELOW CONSIDERS FEAR AND AFFECT TYPE
######################################################################


In [7]:
######################################################################
#plot similarity matrix
######################################################################
#read array output and plot
#load mask in atlas
for mask_n, mask_f in enumerate(mask_fs):
    atlas_img = nib.load(mask_f)
    full_mask = atlas_img.get_fdata()
    num_regions = list(range(1,int(np.max(full_mask))+1))
    mask_labels = pd.read_table(mask_label_fs[mask_n], sep=',', header=None, names = ['index','region_name'], index_col=False)
    
    for mask_num in mask_range:
        atlas_img = nib.load(mask_f)
        mask = atlas_img.get_fdata()
        mask[mask!=int(mask_labels['index'][mask_num])]=0
        mask[mask!=0]=1
        mask_img = nib.Nifti1Image(mask, atlas_img.affine, atlas_img.header)
        mask_label = mask_labels['region_name'][mask_num].replace(" ", "_")
        mask_label = mask_label.replace(",", "_")
        subfolder_name = str(mask_labels['index'][mask_num])+'_'+mask_label

        print(mask_num)
        print(mask_label)

        #load novelty info
        old_vid = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_novelty_array(1-O,0-N).npy'),allow_pickle=True)
        #load affect info
        affect_score = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_normative_highlow_rating_array.npy'),allow_pickle=True)
        #load validity info
        vid_validity = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_validity_array.npy'),allow_pickle=True)        
        #load video name info
        vid_name_orig = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_name_array.npy'),allow_pickle=True)
        vid_name = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_name_array.npy'),allow_pickle=True)
        for dim1 in range(vid_name.shape[0]):
                for dim2 in range(vid_name[dim1].shape[0]):
                    vid_name[dim1][dim2] = vid_name[dim1][dim2].split('_')[0]
 
        #load average timecourse array
        vid_avg_activity_rest_baseline = np.load(os.path.join(save_dir,subfolder_name,subfolder_name+'_timecourse_collapsed_array.npy'),allow_pickle=True)
        #NOTE: crop the data to the period we wanna look at. This matters what we wanna show and how to mark time stamp
        vid_avg_activity_rest_baseline = vid_avg_activity_rest_baseline[TR_BOLD_offset+start_TR:TR_BOLD_offset+start_TR+TR_mean_trial_duration,:,:,:]
        #selection based on validity 
        for dim1 in range(vid_validity.shape[0]):
            for dim2 in range(vid_validity[dim1].shape[0]):
                if vid_validity[dim1][dim2]==0:
                    vid_avg_activity_rest_baseline[:,:,dim1, dim2] = np.nan
                    
        #initiate dataframe with correlation mean
        mean_corr_df = pd.DataFrame(columns = ['mean_corr','subj','novelty','fear_situation','affect','video','ROI'])

        #select based on video name or fear type
        for fearType in ['Social', 'Spiders', 'Heights']:
            print(fearType)

            #select avg signal based on fearType
            vid_avg_activity_rest_baseline_fearType = select4Dfrom2D(vid_avg_activity_rest_baseline, vid_name, fearType)
            #select novelty array based on fearType
            old_vid_fearType = select2Dfrom2D(old_vid, vid_name, fearType)
            #select video name array based on fearType
            vid_name_fearType = select2Dfrom2D(vid_name_orig, vid_name, fearType)
            #select based on novelty
            vid_avg_activity_rest_baseline_fearType_old = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType, old_vid_fearType, 1)
            vid_avg_activity_rest_baseline_fearType_new = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType, old_vid_fearType, 0)

            for affectType in ['high','low']:
                print(affectType)
                
                #further select based on affect
                affect_fearType = select2Dfrom2D(affect_score, vid_name, fearType)
                #select affect array based on novelty and fearType
                affect_OLD_fearType = select2Dfrom2D(affect_fearType, old_vid_fearType, 1)
                affect_NEW_fearType = select2Dfrom2D(affect_fearType, old_vid_fearType, 0)
                #select video name array based on novelty and fearType
                vid_name_fearType_OLD = select2Dfrom2D(vid_name_fearType, old_vid_fearType, 1)
                vid_name_fearType_NEW = select2Dfrom2D(vid_name_fearType, old_vid_fearType, 0)
                #select video name array based on novelty and fearType
                vid_name_fearType_OLD_affect = select2Dfrom2D(vid_name_fearType_OLD, affect_OLD_fearType, affectType)
                vid_name_fearType_NEW_affect = select2Dfrom2D(vid_name_fearType_NEW, affect_NEW_fearType, affectType)
                #select based on affect AND novelty
                vid_avg_activity_rest_baseline_fearType_old_affect = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType_old, affect_OLD_fearType, affectType)
                vid_avg_activity_rest_baseline_fearType_new_affect = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType_new, affect_NEW_fearType, affectType)

                ################################## 
                #plot subject level OLD, NEW, AND CONTRAST similarity matrix cross TR
                ################################## 
                fig, ax = plt.subplots(nrows=len(subj_dir), ncols=3, sharex=True, figsize=(12,200))
                # fig, ax = plt.subplots(nrows=len(subj_dir), ncols=3, sharex=True, figsize=(20,200))
                #         fig, ax = plt.subplots(nrows=len(subj_dir), ncols=4, sharex=True)
                subj_correlation_coe = []
                subj_correlation_coe_new = []
                subj_correlation_coe_old = []

                for n in range(len(subj_dir)):
                    #new
                    temp=[]
                    for v in range(vid_avg_activity_rest_baseline_fearType_new_affect.shape[2]):
                        temp.append(np.corrcoef(vid_avg_activity_rest_baseline_fearType_new_affect[:,:,v,n].astype(float)))
                        #append to mean corr dataframe, this uses shorter TR range
                        corr_mtx = np.corrcoef(vid_avg_activity_rest_baseline_fearType_new_affect[0-start_TR:matrix_average_duration-start_TR,:,v,n].astype(float))
                        corr_mtx[corr_mtx>=0.9999999999]=np.nan
                        mean_corr_df.loc[len(mean_corr_df)]=[np.nanmean(corr_mtx), subj_dir[n], 'new', fearType, affectType, vid_name_fearType_NEW_affect[v][n], mask_label]
                    subj_correlation_coe_new.append(np.nanmean(temp,axis=0))    
                    subj_correlation_coe_new[n][subj_correlation_coe_new[n]>=0.9999999999]=np.nan
                    #old
                    temp=[]
                    for v in range(vid_avg_activity_rest_baseline_fearType_old_affect.shape[2]):
                        temp.append(np.corrcoef(vid_avg_activity_rest_baseline_fearType_old_affect[:,:,v,n].astype(float)))
                        #append to mean corr dataframe, this uses shorter TR range
                        corr_mtx = np.corrcoef(vid_avg_activity_rest_baseline_fearType_old_affect[0-start_TR:matrix_average_duration-start_TR,:,v,n].astype(float))
                        corr_mtx[corr_mtx>=0.9999999999]=np.nan
                        mean_corr_df.loc[len(mean_corr_df)]=[np.nanmean(corr_mtx), subj_dir[n], 'old', fearType, affectType, vid_name_fearType_OLD_affect[v][n], mask_label]
                    subj_correlation_coe_old.append(np.nanmean(temp,axis=0))    
                    subj_correlation_coe_old[n][subj_correlation_coe_old[n]>=0.9999999999]=np.nan
                    #new-old
                    subj_correlation_coe_NEWvOLD = subj_correlation_coe_new[n]-subj_correlation_coe_old[n]

                    ###########NEW#############
                    cax = ax[n,0].matshow(subj_correlation_coe_new[n], cmap='rainbow', extent=matshow_extent)
                    # ax.grid(True)
                    ax[n,0].set_title(subj_dir[n]+': NEW')
                    ax[n,0].set_xticks(matrix_ticks_range);
                    ax[n,0].set_yticks(matrix_ticks_range);
                    #frame for video duration
                    ax[n,0].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                    #frame for rating duratin
                    ax[n,0].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
                    #frame for ITI duratin
                    ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                    ax[n,0].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,0].set_xlabel('TR')
                    ax[n,0].set_ylabel('TR')

                    ###########OLD#############
                    cax = ax[n,1].matshow(subj_correlation_coe_old[n], cmap='rainbow', extent=matshow_extent)
                    # ax.grid(True)
                    ax[n,1].set_title(subj_dir[n]+': OLD')
                    ax[n,1].set_xticks(matrix_ticks_range);
                    ax[n,1].set_yticks(matrix_ticks_range);
                    #frame for video duration
                    ax[n,1].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                    #frame for rating duratin
                    ax[n,1].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
                    #frame for ITI duratin
                    ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                    ax[n,1].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,1].set_xlabel('TR')
                    ax[n,1].set_ylabel('TR')

                    ###########NEW-OLD#############
                    cax = ax[n,2].matshow(subj_correlation_coe_NEWvOLD, cmap='rainbow', extent=matshow_extent)
                    # ax.grid(True)
                    ax[n,2].set_title(subj_dir[n]+': NEW-OLD')
                    ax[n,2].set_xticks(matrix_ticks_range);
                    ax[n,2].set_yticks(matrix_ticks_range);
                    #frame for video duration
                    ax[n,2].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                    #frame for rating duratin
                    ax[n,2].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
                    #frame for ITI duratin
                    ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                    ax[n,2].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                    ax[n,2].set_xlabel('TR')
                    ax[n,2].set_ylabel('TR')

                    fig.colorbar(cax,ax=ax[n,0:3],fraction=0.046, pad=0.04)

                plt.savefig(os.path.join(save_dir,subfolder_name,subfolder_name+'_sim_matrix_subj_'+fearType+'_'+affectType+'Affect.png'), bbox_inches = 'tight',pad_inches = 0.1)
                mean_corr_df.to_csv(os.path.join(save_dir,subfolder_name,subfolder_name+'_mean_corr.csv'))

                ################################## 
                #plot mean similarity matrix cross TR OLD OR NEW ONLY
                ################################## 
                fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(14,7))

                #NEW
                mean_correlation_coe_new = np.nanmean(subj_correlation_coe_new, axis=0)

                cax = ax[0].matshow(mean_correlation_coe_new, cmap='rainbow', extent=matshow_extent)
                ax[0].set_title('NEW: similarity matrix of averaged BOLD signal change of '+fearType+' '+affectType+' affect video')
                ax[0].set_xticks(matrix_ticks_range);
                ax[0].set_yticks(matrix_ticks_range);
                #frame for video duration
                ax[0].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                ax[0].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
                ax[0].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                #frame for rating duratin
                ax[0].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
                #frame for ITI duratin
                ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                ax[0].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)

                ax[0].set_xlabel('TR')
                ax[0].set_ylabel('TR')
                # fig.colorbar(cax,ax=ax[0],fraction=0.046, pad=0.04, label = 'correlation coefficient r')

                #OLD
                mean_correlation_coe_old = np.nanmean(subj_correlation_coe_old, axis=0)

                cax = ax[1].matshow(mean_correlation_coe_old, cmap='rainbow', extent=matshow_extent)
                ax[1].set_title('OLD: '+fearType+' '+affectType+' affect video in '+mask_labels['region_name'][mask_num])
                ax[1].set_xticks(matrix_ticks_range);
                ax[1].set_yticks(matrix_ticks_range);
                #frame for video duration
                ax[1].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                ax[1].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
                ax[1].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
                #frame for rating duratin
                ax[1].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
                #frame for ITI duratin
                ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
                ax[1].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)

                ax[1].set_xlabel('TR')
                ax[1].set_ylabel('TR')
                fig.colorbar(cax,ax=ax.ravel().tolist(),shrink=0.75, label = 'correlation coefficient r')

                plt.savefig(os.path.join(save_dir,subfolder_name,subfolder_name+'_sim_matrix_NEWnOLD_'+fearType+'_'+affectType+'Affect.png'), bbox_inches = 'tight',pad_inches = 0.1)

                colorbar_range_min = np.nanmin([np.nanmin(mean_correlation_coe_new),np.nanmin(mean_correlation_coe_old)])
                colorbar_range_max = np.nanmax([np.nanmax(mean_correlation_coe_new),np.nanmax(mean_correlation_coe_old)])

                ################################## 
                #OLD vs NEW contrast
                ################################## 
                subj_correlation_coe_NEWvOLD = [x-y for x,y in zip(subj_correlation_coe_new,subj_correlation_coe_old)]
                mean_correlation_coe_NEWvOLD = np.nanmean(np.array(subj_correlation_coe_NEWvOLD), axis=0)

                fig, ax = plt.subplots(figsize=(7,7))
                # cax = ax.matshow(mean_correlation_coe_NEWvOLD, vmin=colorbar_range_min, vmax=colorbar_range_max,  cmap='rainbow')
                cax = ax.matshow(mean_correlation_coe_NEWvOLD,  cmap='rainbow', extent=matshow_extent)
                # ax.grid(True)
                plt.title('NEW-OLD similarity matrix of averaged BOLD signal change of '+fearType+' '+affectType+' affect video in '+mask_labels['region_name'][mask_num])
                plt.xticks(matrix_ticks_range, rotation=90);
                plt.yticks(matrix_ticks_range);
                #0.046 and 0.04 are magical numbers. Don't think too much
                plt.colorbar(cax,fraction=0.046, pad=0.04, label = 'correlation coefficient r')

                #frame for video duration
                plt.plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=3)
                plt.plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=3)
                plt.plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=3)
                #frame for rating duratin
                plt.plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=3)
                #frame for ITI duratin
                plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=3)
                plt.plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)

                #paired t-test on every TR
                p_value = stats.ttest_rel(subj_correlation_coe_new,subj_correlation_coe_old, nan_policy='omit')[1]
                for x in range(p_value.shape[0]):
                    for y in range(p_value.shape[1]):
                        if p_value[x][y] < 0.05:
                            plt.plot((x+start_TR, x+start_TR), (y+start_TR, y+start_TR+1), '-', color = 'black',linewidth=1.1)
                            plt.plot((x+start_TR+1, x+start_TR), (y+start_TR+1, y+start_TR+1), '-', color = 'black',linewidth=1.1)
                            plt.plot((x+start_TR+1, x+start_TR), (y+start_TR, y+start_TR), '-', color = 'black',linewidth=1.1)
                            plt.plot((x+start_TR+1, x+start_TR+1), (y+start_TR+1, y+start_TR), '-', color = 'black',linewidth=1.1)
                #                     plt.scatter(x, y, color = 'white',s=2)

                plt.xlabel('TR')
                plt.ylabel('TR')

                plt.savefig(os.path.join(save_dir,subfolder_name,subfolder_name+'_sim_matrix_NEWvOLD_'+fearType+'_'+affectType+'Affect.png'), bbox_inches = 'tight',pad_inches = 0.1)


2
superior_frontal_gyrus
Social
high
low
Spiders
high
low
Heights
high
low
4
inferior_frontal_gyrus__pars_triangularis
Social
high
low
Spiders
high
low
Heights
high
low
5
inferior_frontal_gyrus__pars_opercularis
Social
high
low
Spiders
high
low
Heights
high
low
6
precentral_gyrus
Social
high
low
Spiders
high
low
Heights
high
low
7
temporal_pole
Social
high
low
Spiders
high
low
Heights
high
low
8
superior_temporal_gyrus__anterior_division
Social
high
low
Spiders
high
low
Heights
high
low
9
superior_temporal_gyrus__posterior_division
Social
high
low
Spiders
high
low
Heights
high
low
10
middle_temporal_gyrus__anterior_division
Social
high
low
Spiders
high
low
Heights
high
low
11
middle_temporal_gyrus__posterior_division
Social
high
low
Spiders
high
low
Heights
high
low


######################################################################

CODE BELOW DOESN'T CONSIDER FEAR AND AFFECT TYPE
######################################################################


In [ ]:
######################################################################
#plot similarity matrix
######################################################################
#read array output and plot
#load mask in atlas
for mask_n, mask_f in enumerate(mask_fs):
    atlas_img = nib.load(mask_f)
    full_mask = atlas_img.get_fdata()
    num_regions = list(range(1,int(np.max(full_mask))+1))
    mask_labels = pd.read_table(mask_label_fs[mask_n], sep=',', header=None, names = ['index','region_name'], index_col=False)
    
    for mask_num in mask_range:
        atlas_img = nib.load(mask_f)
        mask = atlas_img.get_fdata()
        mask[mask!=int(mask_labels['index'][mask_num])]=0
        mask[mask!=0]=1
        mask_img = nib.Nifti1Image(mask, atlas_img.affine, atlas_img.header)
        mask_label = mask_labels['region_name'][mask_num].replace(" ", "_")
        mask_label = mask_label.replace(",", "_")
        subfolder_name = str(mask_labels['index'][mask_num])+'_'+mask_label

        print(mask_label)

        #load novelty info
        old_vid = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_novelty_array(1-O,0-N).npy'),allow_pickle=True)
        #load affect info
        affect_score = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_normative_highlow_rating_array.npy'),allow_pickle=True)
        #load validity info
        vid_validity = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_validity_array.npy'),allow_pickle=True)        
        #load video name info
        vid_name_orig = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_name_array.npy'),allow_pickle=True)
        vid_name = np.load(os.path.join(save_dir,'subj_video_info','subj_vid_name_array.npy'),allow_pickle=True)
        for dim1 in range(vid_name.shape[0]):
                for dim2 in range(vid_name[dim1].shape[0]):
                    vid_name[dim1][dim2] = vid_name[dim1][dim2].split('_')[0]
 
        #load average timecourse array
        vid_avg_activity_rest_baseline = np.load(os.path.join(save_dir,subfolder_name,subfolder_name+'_timecourse_collapsed_array.npy'),allow_pickle=True)
        #NOTE: crop the data to the period we wanna look at. This matters what we wanna show and how to mark time stamp
        vid_avg_activity_rest_baseline = vid_avg_activity_rest_baseline[TR_BOLD_offset+start_TR:TR_BOLD_offset+start_TR+TR_mean_trial_duration,:,:,:]
        #selection based on validity 
        for dim1 in range(vid_validity.shape[0]):
            for dim2 in range(vid_validity[dim1].shape[0]):
                if vid_validity[dim1][dim2]==0:
                    vid_avg_activity_rest_baseline[:,:,dim1, dim2] = np.nan
                    
#         #initiate dataframe with correlation mean
#         mean_corr_df = pd.DataFrame(columns = ['mean_corr','subj','novelty','fear_situation','affect','video','ROI'])

        #select based on novelty
        vid_avg_activity_rest_baseline_old = select4Dfrom2D(vid_avg_activity_rest_baseline, old_vid, 1)
        vid_avg_activity_rest_baseline_new = select4Dfrom2D(vid_avg_activity_rest_baseline, old_vid, 0)

#         #select based on video name or fear type
#         for fearType in ['Social', 'Spiders', 'Heights']:
#             print(fearType)
            
#             #select avg signal based on fearType
#             vid_avg_activity_rest_baseline_fearType = select4Dfrom2D(vid_avg_activity_rest_baseline, vid_name, fearType)
#             #select novelty array based on fearType
#             old_vid_fearType = select2Dfrom2D(old_vid, vid_name, fearType)

#             #select avg signal based on fearType
#             vid_avg_activity_rest_baseline_fearType = select4Dfrom2D(vid_avg_activity_rest_baseline, vid_name, fearType)
#             #select novelty array based on fearType
#             old_vid_fearType = select2Dfrom2D(old_vid, vid_name, fearType)
#             #select video name array based on fearType
#             vid_name_fearType = select2Dfrom2D(vid_name_orig, vid_name, fearType)
#             #select based on novelty
#             vid_avg_activity_rest_baseline_fearType_old = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType, old_vid_fearType, 1)
#             vid_avg_activity_rest_baseline_fearType_new = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType, old_vid_fearType, 0)

#             for affectType in ['high','low']:
#                 print(affectType)
                
#                 #further select based on affect
#                 affect_fearType = select2Dfrom2D(affect_score, vid_name, fearType)
#                 #select affect array based on novelty and fearType
#                 affect_OLD_fearType = select2Dfrom2D(affect_fearType, old_vid_fearType, 1)
#                 affect_NEW_fearType = select2Dfrom2D(affect_fearType, old_vid_fearType, 0)
#                 #select video name array based on novelty and fearType
#                 vid_name_fearType_OLD = select2Dfrom2D(vid_name_fearType, old_vid_fearType, 1)
#                 vid_name_fearType_NEW = select2Dfrom2D(vid_name_fearType, old_vid_fearType, 0)
#                 #select video name array based on novelty and fearType
#                 vid_name_fearType_OLD_affect = select2Dfrom2D(vid_name_fearType_OLD, affect_OLD_fearType, affectType)
#                 vid_name_fearType_NEW_affect = select2Dfrom2D(vid_name_fearType_NEW, affect_NEW_fearType, affectType)
#                 #select based on affect AND novelty
#                 vid_avg_activity_rest_baseline_fearType_old_affect = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType_old, affect_OLD_fearType, affectType)
#                 vid_avg_activity_rest_baseline_fearType_new_affect = select4Dfrom2D(vid_avg_activity_rest_baseline_fearType_new, affect_NEW_fearType, affectType)

        ################################## 
        #plot subject level OLD, NEW, AND CONTRAST similarity matrix cross TR
        ################################## 
        fig, ax = plt.subplots(nrows=len(subj_dir), ncols=3, sharex=True, figsize=(12,200))
        # fig, ax = plt.subplots(nrows=len(subj_dir), ncols=3, sharex=True, figsize=(20,200))
        #         fig, ax = plt.subplots(nrows=len(subj_dir), ncols=4, sharex=True)
        subj_correlation_coe = []
        subj_correlation_coe_new = []
        subj_correlation_coe_old = []

        for n in range(len(subj_dir)):
            #new
            temp=[]
            for v in range(vid_avg_activity_rest_baseline_new.shape[2]):
                temp.append(np.corrcoef(vid_avg_activity_rest_baseline_new[:,:,v,n].astype(float)))
#                 #append to mean corr dataframe, this uses shorter TR range
#                 corr_mtx = np.corrcoef(vid_avg_activity_rest_baseline_fearType_new_affect[0-start_TR:matrix_average_duration-start_TR,:,v,n].astype(float))
#                 corr_mtx[corr_mtx>=0.9999999999]=np.nan
#                 mean_corr_df.loc[len(mean_corr_df)]=[np.nanmean(corr_mtx), subj_dir[n], 'new', fearType, affectType, vid_name_fearType_NEW_affect[v][n], mask_label]
            subj_correlation_coe_new.append(np.nanmean(temp,axis=0))    
            subj_correlation_coe_new[n][subj_correlation_coe_new[n]>=0.9999999999]=np.nan
            #old
            temp=[]
            for v in range(vid_avg_activity_rest_baseline_old.shape[2]):
                temp.append(np.corrcoef(vid_avg_activity_rest_baseline_old[:,:,v,n].astype(float)))
#                 #append to mean corr dataframe, this uses shorter TR range
#                 corr_mtx = np.corrcoef(vid_avg_activity_rest_baseline_fearType_old_affect[0-start_TR:matrix_average_duration-start_TR,:,v,n].astype(float))
#                 corr_mtx[corr_mtx>=0.9999999999]=np.nan
#                 mean_corr_df.loc[len(mean_corr_df)]=[np.nanmean(corr_mtx), subj_dir[n], 'old', fearType, affectType, vid_name_fearType_OLD_affect[v][n], mask_label]
            subj_correlation_coe_old.append(np.nanmean(temp,axis=0))    
            subj_correlation_coe_old[n][subj_correlation_coe_old[n]>=0.9999999999]=np.nan
            #new-old
            subj_correlation_coe_NEWvOLD = subj_correlation_coe_new[n]-subj_correlation_coe_old[n]

            ###########NEW#############
            cax = ax[n,0].matshow(subj_correlation_coe_new[n], cmap='rainbow', extent=matshow_extent)
            # ax.grid(True)
            ax[n,0].set_title(subj_dir[n]+': NEW')
            ax[n,0].set_xticks(matrix_ticks_range);
            ax[n,0].set_yticks(matrix_ticks_range);
            #frame for video duration
            ax[n,0].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,0].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
            ax[n,0].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
            #frame for rating duratin
            ax[n,0].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
            #frame for ITI duratin
            ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
            ax[n,0].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,0].set_xlabel('TR')
            ax[n,0].set_ylabel('TR')

            ###########OLD#############
            cax = ax[n,1].matshow(subj_correlation_coe_old[n], cmap='rainbow', extent=matshow_extent)
            # ax.grid(True)
            ax[n,1].set_title(subj_dir[n]+': OLD')
            ax[n,1].set_xticks(matrix_ticks_range);
            ax[n,1].set_yticks(matrix_ticks_range);
            #frame for video duration
            ax[n,1].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,1].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
            ax[n,1].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
            #frame for rating duratin
            ax[n,1].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
            #frame for ITI duratin
            ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
            ax[n,1].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,1].set_xlabel('TR')
            ax[n,1].set_ylabel('TR')

            ###########NEW-OLD#############
            cax = ax[n,2].matshow(subj_correlation_coe_NEWvOLD, cmap='rainbow', extent=matshow_extent)
            # ax.grid(True)
            ax[n,2].set_title(subj_dir[n]+': NEW-OLD')
            ax[n,2].set_xticks(matrix_ticks_range);
            ax[n,2].set_yticks(matrix_ticks_range);
            #frame for video duration
            ax[n,2].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,2].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
            ax[n,2].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
            #frame for rating duratin
            ax[n,2].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
            #frame for ITI duratin
            ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
            ax[n,2].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
            ax[n,2].set_xlabel('TR')
            ax[n,2].set_ylabel('TR')

            fig.colorbar(cax,ax=ax[n,0:3],fraction=0.046, pad=0.04)

        plt.savefig(os.path.join(save_dir,subfolder_name,subfolder_name+'_sim_matrix_subj.png'), bbox_inches = 'tight',pad_inches = 0.1)
#         mean_corr_df.to_csv(os.path.join(save_dir,subfolder_name,subfolder_name+'_mean_corr.csv'))

        ################################## 
        #plot mean similarity matrix cross TR OLD OR NEW ONLY
        ################################## 
        fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(14,7))

        #NEW
        mean_correlation_coe_new = np.nanmean(subj_correlation_coe_new, axis=0)

        cax = ax[0].matshow(mean_correlation_coe_new, cmap='rainbow', extent=matshow_extent)
        ax[0].set_title('NEW: similarity matrix of averaged BOLD signal change')
        ax[0].set_xticks(matrix_ticks_range);
        ax[0].set_yticks(matrix_ticks_range);
        #frame for video duration
        ax[0].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
        ax[0].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
        ax[0].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
        #frame for rating duratin
        ax[0].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
        #frame for ITI duratin
        ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
        ax[0].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)

        ax[0].set_xlabel('TR')
        ax[0].set_ylabel('TR')
        # fig.colorbar(cax,ax=ax[0],fraction=0.046, pad=0.04, label = 'correlation coefficient r')

        #OLD
        mean_correlation_coe_old = np.nanmean(subj_correlation_coe_old, axis=0)

        cax = ax[1].matshow(mean_correlation_coe_old, cmap='rainbow', extent=matshow_extent)
        ax[1].set_title('OLD: video in '+mask_labels['region_name'][mask_num])
        ax[1].set_xticks(matrix_ticks_range);
        ax[1].set_yticks(matrix_ticks_range);
        #frame for video duration
        ax[1].plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=2)
        ax[1].plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=2)
        ax[1].plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=2)
        #frame for rating duratin
        ax[1].plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=2)
        #frame for ITI duratin
        ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=2)
        ax[1].plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=2)

        ax[1].set_xlabel('TR')
        ax[1].set_ylabel('TR')
        fig.colorbar(cax,ax=ax.ravel().tolist(),shrink=0.75, label = 'correlation coefficient r')

        plt.savefig(os.path.join(save_dir,subfolder_name,subfolder_name+'_sim_matrix_NEWnOLD.png'), bbox_inches = 'tight',pad_inches = 0.1)

        colorbar_range_min = np.nanmin([np.nanmin(mean_correlation_coe_new),np.nanmin(mean_correlation_coe_old)])
        colorbar_range_max = np.nanmax([np.nanmax(mean_correlation_coe_new),np.nanmax(mean_correlation_coe_old)])

        ################################## 
        #OLD vs NEW contrast
        ################################## 
        subj_correlation_coe_NEWvOLD = [x-y for x,y in zip(subj_correlation_coe_new,subj_correlation_coe_old)]
        mean_correlation_coe_NEWvOLD = np.nanmean(np.array(subj_correlation_coe_NEWvOLD), axis=0)

        fig, ax = plt.subplots(figsize=(7,7))
        # cax = ax.matshow(mean_correlation_coe_NEWvOLD, vmin=colorbar_range_min, vmax=colorbar_range_max,  cmap='rainbow')
        cax = ax.matshow(mean_correlation_coe_NEWvOLD,  cmap='rainbow', extent=matshow_extent)
        # ax.grid(True)
        plt.title('NEW-OLD similarity matrix of averaged BOLD signal change of video in '+mask_labels['region_name'][mask_num])
        plt.xticks(matrix_ticks_range, rotation=90);
        plt.yticks(matrix_ticks_range);
        #0.046 and 0.04 are magical numbers. Don't think too much
        plt.colorbar(cax,fraction=0.046, pad=0.04, label = 'correlation coefficient r')

        #frame for video duration
        plt.plot((0, 0), (0, TR_video_duration), '--', color = 'white',linewidth=3)
        plt.plot((0, TR_video_duration), (0, 0), '--', color = 'white',linewidth=3)
        plt.plot((0, TR_video_duration), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration, TR_video_duration), (0, TR_video_duration), '--', color = 'white',linewidth=3)
        #frame for rating duratin
        plt.plot((TR_video_duration, TR_video_duration), (TR_video_duration, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration, TR_video_duration), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration), '--', color = 'white',linewidth=3)
        #frame for ITI duratin
        plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration+TR_mean_rating, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), '--', color = 'white',linewidth=3)
        plt.plot((TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating+TR_mean_mean_ITI), (TR_video_duration+TR_mean_rating+TR_mean_mean_ITI, TR_video_duration+TR_mean_rating), '--', color = 'white',linewidth=3)

        #paired t-test on every TR
        p_value = stats.ttest_rel(subj_correlation_coe_new,subj_correlation_coe_old, nan_policy='omit')[1]
        for x in range(p_value.shape[0]):
            for y in range(p_value.shape[1]):
                if p_value[x][y] < 0.05:
                    plt.plot((x+start_TR, x+start_TR), (y+start_TR, y+start_TR+1), '-', color = 'black',linewidth=1.1)
                    plt.plot((x+start_TR+1, x+start_TR), (y+start_TR+1, y+start_TR+1), '-', color = 'black',linewidth=1.1)
                    plt.plot((x+start_TR+1, x+start_TR), (y+start_TR, y+start_TR), '-', color = 'black',linewidth=1.1)
                    plt.plot((x+start_TR+1, x+start_TR+1), (y+start_TR+1, y+start_TR), '-', color = 'black',linewidth=1.1)
        #                     plt.scatter(x, y, color = 'white',s=2)

        plt.xlabel('TR')
        plt.ylabel('TR')

        plt.savefig(os.path.join(save_dir,subfolder_name,subfolder_name+'_sim_matrix_NEWvOLD.png'), bbox_inches = 'tight',pad_inches = 0.1)


frontal_pole


In [ ]:
x